In [3]:
# Dependencies
import pandas as pd
import numpy as np

import pymc as pm
import arviz as az

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt

In [14]:
# Load data
data = pd.read_csv('transformed_data.csv', index_col=0, parse_dates=True)
# Set frequency as monthly
data = data.asfreq('MS').dropna()

# Select a subset of indicators most correlated with CPI
FEATURES = 13
LAGS = 6
corr = data.corr()
cpi_corr = corr['CPIAUCSL'].sort_values(ascending=False)
# Select most correlated columns
top_corr = cpi_corr.index[0: FEATURES]
data = data[top_corr]

# Prepare lagged data
Y = data.iloc[LAGS:]  # Target variables
X = np.hstack([data.shift(i).iloc[LAGS:].values for i in range(1, LAGS + 1)])  # Lagged predictors

In [15]:
with pm.Model() as bvar_model:
    # Priors for autoregressive coefficients (Normal around 0 with shrinkage)
    beta = pm.Normal("beta", mu=0, sigma=0.1, shape=(X.shape[1], Y.shape[1]))

    # Prior for error variance (Inverse Gamma)
    sigma = pm.HalfCauchy("sigma", beta=1, shape=Y.shape[1])

    # Likelihood: Linear regression model
    mu = pm.math.dot(X, beta)  # Linear combination of lags
    Y_obs = pm.Normal("Y_obs", mu=mu, sigma=sigma, observed=Y.values)

    # Sampling
    trace = pm.sample(2000, tune=1000, target_accept=0.9)


Initializing NUTS using jitter+adapt_diag...
/home/js/macroeconvue/modeling/.venv/lib/python3.12/site-packages/pytensor/link/c/cmodule.py:2959: UserWarning: PyTensor could not link to a BLAS installation. Operations that might benefit from BLAS will be severely degraded.
This usually happens when PyTensor is installed via pip. We recommend it be installed via conda/mamba/pixi instead.
Alternatively, you can use an experimental backend such as Numba or JAX that perform their own BLAS optimizations, by setting `pytensor.config.mode == 'NUMBA'` or passing `mode='NUMBA'` when compiling a PyTensor function.
For more options and details see https://pytensor.readthedocs.io/en/latest/troubleshooting.html#how-do-i-configure-test-my-blas-library
  warnings.warn(
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta,

/home/js/macroeconvue/modeling/.venv/lib/python3.12/site-packages/rich/live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 73 seconds.


In [16]:
def forecast_bvar(trace, X, steps=6):
    posterior_samples = trace.posterior["beta"].mean(dim=("chain", "draw")).values
    sigma_samples = trace.posterior["sigma"].mean(dim=("chain", "draw")).values
    
    predictions = []
    X_pred = X[-1]  # Last observed lag

    for _ in range(steps):
        Y_next = X_pred @ posterior_samples
        noise = np.random.normal(0, sigma_samples, size=Y_next.shape)
        Y_next += noise
        predictions.append(Y_next)
        
        # Update lags
        X_pred = np.roll(X_pred, -Y.shape[1])
        X_pred[-Y.shape[1]:] = Y_next

    return np.array(predictions)

# Generate forecasts
forecasts = forecast_bvar(trace, X, steps=6)

# Convert forecasts to DataFrame
forecast_df = pd.DataFrame(forecasts, columns=Y.columns)
print(forecast_df)


   CPIAUCSL  CUSR0000SA0L5  CPIULFSL  CUSR0000SA0L2  CUSR0000SAC  CPITRNSL  \
0  0.002119       0.001895 -0.002813       0.000011    -0.007568 -0.037994   
1 -0.001863       0.004030  0.001474      -0.004188     0.011273 -0.009759   
2  0.002442       0.004827  0.003487       0.001666    -0.001809  0.002692   
3  0.003839       0.003627  0.009044       0.004256     0.002787  0.006766   
4  0.000408       0.004089  0.007394       0.007796     0.014468  0.014260   
5  0.000539       0.002112  0.001529      -0.003344     0.007769 -0.000203   

   DNDGRG3M086SBEA     PCEPI  WPSFD49502  WPSFD49207   WPSID61    ACOGNO  \
0        -0.004805  0.003070    0.023015    0.007769  0.011980 -0.038344   
1        -0.006123 -0.000102   -0.013829   -0.003693 -0.003774 -0.028020   
2         0.015036  0.001166    0.013260    0.000675 -0.000930 -0.059722   
3        -0.002999 -0.000568    0.009272   -0.004171  0.002789 -0.040686   
4         0.012572  0.001939    0.029481    0.005799  0.007249  0.007789 

In [23]:
# Calculate normalized RMSE of CPIAUCSL
rmse = np.sqrt(mean_squared_error(data['CPIAUCSL'].iloc[-6:].values, forecast_df['CPIAUCSL']))
print(f"{rmse}, {rmse / data['CPIAUCSL'].iloc[-6:].mean() * 100:.2f}\n")

0.00261909888837995, -2288.54

